In [1]:
!apt-get install libmagic-dev
!pip install python-magic

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libmagic-dev is already the newest version (1:5.32-2ubuntu0.4).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.


In [2]:
!pip install secml_malware

In [3]:
!pip install git+https://github.com/elastic/ember.git

  Cloning https://github.com/elastic/ember.git to /tmp/pip-req-build-iqjgryub
  Running command git clone -q https://github.com/elastic/ember.git /tmp/pip-req-build-iqjgryub


In [4]:
import os
import magic
import secml_malware
from secml.array import CArray

from secml_malware.attack.whitebox import CKreukEvasion
from secml_malware.models.malconv import MalConv
from secml_malware.models.c_classifier_end2end_malware import CClassifierEnd2EndMalware, End2EndModel

net = MalConv()
net = CClassifierEnd2EndMalware(net)
net.load_pretrained_model()

In [5]:
from secml_malware.attack.whitebox.c_header_evasion import CHeaderEvasion

partial_dos = CHeaderEvasion(net, random_init=False, iterations=50, optimize_all_dos=False, threshold=0.5)

In [6]:
folder = "/content/drive/MyDrive/SUL lab/SUL_Package"
X = []
y = []
file_names = []
for i, f in enumerate(os.listdir(folder)):
   continue

In [7]:
i= enumerate(os.listdir(folder))
print(f)

spam_ham_dataset.csv


In [8]:
os.listdir(folder)

['spam_ham_dataset.csv']

In [13]:
path=os.path.join(folder, f)

In [14]:
magic.from_file(path)

'data'

In [15]:
open(path, "rb")

<_io.BufferedReader name='/content/drive/MyDrive/SUL lab/SUL_Package/spam_ham_dataset.csv'>

In [16]:
with open(path, "rb") as file_handle:
    code = file_handle.read()
    x = End2EndModel.bytes_to_numpy(code, net.get_input_max_length(), 256, False)
    _, confidence = net.predict(CArray(x), True)
    print(x)
    print(confidence)

[ 44. 108.  97. ...  32.  59.  32.]
CArray([[9.999966e-01 3.439687e-06]])


In [17]:
print(f"> Added {f} with confidence {confidence[0,1].item()}")
X.append(x)

> Added spam_ham_dataset.csv with confidence 3.439686679485021e-06


In [18]:
conf = confidence[1][0].item()
y.append([1 - conf, conf])
file_names.append(path)

In [19]:
X

[array([ 44., 108.,  97., ...,  32.,  59.,  32.])]

In [20]:
y

[[0.9999965603133205, 3.439686679485021e-06]]

In [21]:
#adv_ds=
for sample, label in zip(X, y):
    y_pred, adv_score, adv_ds, f_obj = partial_dos.run(CArray(sample), CArray(label[1]))
    print(partial_dos.confidences_)
    print(f_obj)

[3.439686679485021e-06]
3.439686679485021e-06


In [22]:
adv_x = adv_ds.X[0,:]
real_adv_x = partial_dos.create_real_sample_from_adv(file_names[0], adv_x)
print(len(real_adv_x))
real_x = End2EndModel.bytes_to_numpy(real_adv_x, net.get_input_max_length(), 256, False)
_, confidence = net.predict(CArray(real_x), True)
print(confidence[0,1].item())

5502589
3.439686679485021e-06
